# 1. Install AG2 + Materials Project dependencies

In [9]:
!pip install -U "ag2[openai]" autogen -q

import sys
!{sys.executable} -m pip install pymatgen mp-api numpy cython ipywidgets jupyterlab_widgets -q

print("✓ AG2/autogen + Pymatgen + MP-API installed.")


✓ AG2/autogen + Pymatgen + MP-API installed.


# 2. Imports + LLM Configuration


In [10]:

# Standard library imports

import os
import json
import random
from datetime import datetime, timedelta
from typing import Annotated, Any, Literal, Union, List, Optional, Tuple


# AG2 core imports

from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import ReplyResult
from autogen.agentchat.group import (
    ContextVariables,
    AgentTarget, AgentNameTarget, StayTarget,
    OnCondition, StringLLMCondition,
    OnContextCondition, ExpressionContextCondition, ContextExpression,
    RevertToUserTarget, TerminateTarget,
)
from autogen.agentchat.groupchat import GroupChat, GroupChatManager
from autogen.tools import tool


# Tools + External APIs

from mp_api.client import MPRester


# Coding tools

from autogen.coding.local_commandline_code_executor import LocalCommandLineCodeExecutor
from autogen.coding.base import CodeBlock
from pydantic import BaseModel, Field


# LLM configuration

llm_config = LLMConfig(config_list={
    "api_type": "openai",
    "model": "gpt-4o",
    "api_key": os.environ["OPENAI_API_KEY"],
})

print("✓ Imports loaded and LLM configured.")


✓ Imports loaded and LLM configured.


In [11]:
import autogen
print(autogen.__version__)


0.10.1


# 3. User query + Context Variables

In [16]:
from autogen.agentchat.group import ContextVariables

# Development mode switch
DEV_MODE = False   # Set to False to enable real user input

if DEV_MODE:
    # No input while developing
    user_query = "test query"
else:
    # Real interactive input (final version)
    user_query = input("Enter your materials design challenge: ")

# Initialize context variables
context_variables = ContextVariables(
    data={
        "user_query": user_query,
        "explained_terms": None,
        "mp_query": None,
        "mp_results": None,
        "final_conclusion": None,
    }
)

print("✓ Context variables initialized.")



Enter your materials design challenge:  fire mountain forest


✓ Context variables initialized.


# 4. TOOLS (A → B → C → D)

In [25]:
# Tool A — explain_query

@tool(
    name="explain_query_tool",
    description="Store the user query and its explanation in the context, then route to AgentB_MaterialsRetriever."
)
def explain_query_tool(
    query: Annotated[str, "initial query AS provided by the user without modification."],
    query_explanation: Annotated[str, "An explanation of the query, including a breakdown of key and secondary terms"],
    context_variables: ContextVariables,
) -> ReplyResult:

    error_target = AgentNameTarget("AgentA_Explainer")

    # Validation
    try:
        if not isinstance(query, str):
            raise ValueError("query must be a string.")
        if not isinstance(query_explanation, str):
            raise ValueError("query_explanation must be a string.")
        if not query.strip():
            raise ValueError("query cannot be empty.")
        if not query_explanation.strip():
            raise ValueError("query_explanation cannot be empty.")
    except ValueError as e:
        return ReplyResult(
            message=f"Error in explain_query_tool: {e}",
            target=error_target,
            context_variables=context_variables,
        )

    # Update context
    context_variables["task_started"] = True
    context_variables["user_query"] = query
    context_variables["explained_terms"] = query_explanation

    # Logging output
    message = (
        "Scientific explanation of the user query:\n\n"
        f"{query_explanation}"
    )

    return ReplyResult(
        message=message,
        target=AgentNameTarget("AgentB_MaterialsRetriever"),
        context_variables=context_variables,
    )



# Tool B — download_materials_structures_properties_from_mp

@tool(
    name="download_materials_structures_properties_from_mp",
    description="Retrieve materials from Materials Project according to structured criteria."
)
def download_materials_structures_properties_from_mp(
    search_criteria: Annotated[dict, "Filtering conditions used to select candidate materials."],
    fields: Annotated[List[str], "List of metadata fields to retrieve for each material."],
    sample_number: Annotated[Optional[int], "Number of materials to sample. Defaults to 20 if omitted."],
    context_variables: ContextVariables,
) -> ReplyResult:

    error_target = AgentNameTarget("AgentB_MaterialsRetriever")


    # Validation

    try:
        # search_criteria must be non-empty dict
        if not isinstance(search_criteria, dict):
            raise ValueError("search_criteria must be a dictionary.")
        if len(search_criteria) == 0:
            raise ValueError("search_criteria cannot be empty.")

        # fields must be non-empty list of strings
        if not isinstance(fields, list):
            raise ValueError("fields must be a list.")
        if len(fields) == 0:
            raise ValueError("fields list cannot be empty.")
        if not all(isinstance(f, str) for f in fields):
            raise ValueError("each element in fields must be a string.")

        # sample_number validation + fallback
        if sample_number is None:
            sample_number = 20
        if not isinstance(sample_number, int) or sample_number <= 0:
            raise ValueError("sample_number must be a positive integer.")

        # Allowed filters according to MIT mentor spec
        allowed_numeric_filters = {
            "band_gap",
            "energy_above_hull",
            "k_voigt",
            "g_voigt",
            "num_sites",
        }

        allowed_composition_filters = {
            "chemsys",
            "elements",
            "excluded_elements",
        }

        allowed_keys = allowed_numeric_filters | allowed_composition_filters

        # Aliases for MP Summary API
        alias_map = {
            "num_sites": "nsites",
            "k_voigt": "bulk_modulus",
            "g_voigt": "shear_modulus",
        }

    except ValueError as e:
        return ReplyResult(
            message=f"Error in download_materials_structures_properties_from_mp: {e}",
            target=error_target,
            context_variables=context_variables,
        )

   
    # Convert search_criteria → MP API filters
  
    search_kwargs = {}

    try:
        for key, value in search_criteria.items():

            if key not in allowed_keys:
                raise ValueError(
                    f"invalid filter '{key}'. Allowed filters: {sorted(allowed_keys)}"
                )

            internal_key = alias_map.get(key, key)

            # Numeric filters → tuple(min,max)
            if key in allowed_numeric_filters:
                if not (isinstance(value, tuple) and len(value) == 2):
                    raise ValueError(
                        f"Numeric filter '{key}' must be a tuple (min, max)."
                    )

                min_val, max_val = value

                if min_val is not None and not isinstance(min_val, (int, float)):
                    raise ValueError(f"Lower bound of '{key}' must be number or None.")
                if max_val is not None and not isinstance(max_val, (int, float)):
                    raise ValueError(f"Upper bound of '{key}' must be number or None.")

                search_kwargs[internal_key] = (min_val, max_val)

            # Composition filters → list[str]
            else:
                if not isinstance(value, list) or not all(isinstance(x, str) for x in value):
                    raise ValueError(
                        f"Filter '{key}' must be a list of strings."
                    )
                search_kwargs[internal_key] = value

    except ValueError as e:
        return ReplyResult(
            message=f"Error in download_materials_structures_properties_from_mp: {e}",
            target=error_target,
            context_variables=context_variables,
        )

 
    # Load MP API key
  
    api_key = os.getenv("MP_API_KEY")
    if not api_key:
        return ReplyResult(
            message="Error in download_materials_structures_properties_from_mp: MP_API_KEY not set.",
            target=error_target,
            context_variables=context_variables,
        )

 
    # Query Materials Project
 
    try:
        with MPRester(api_key) as mpr:
            all_results = list(
                mpr.materials.summary.search(
                    fields=fields,
                    **search_kwargs,
                )
            )
    except Exception as e:
        return ReplyResult(
            message=f"Error querying Materials Project: {e}",
            target=error_target,
            context_variables=context_variables,
        )

    # Empty set handling
    if len(all_results) == 0:
        return ReplyResult(
            message="Error in download_materials_structures_properties_from_mp: No materials found.",
            target=error_target,
            context_variables=context_variables,
        )


    # Sampling according to sample_number
 
    import random

    if len(all_results) > sample_number:
        results = random.sample(all_results, sample_number)
    else:
        results = all_results


    # Update context

    context_variables["mp_query"] = {
        "search_criteria": search_criteria,
        "fields": fields,
        "sample_number": sample_number,
    }

    context_variables["mp_results"] = results


    # Success → pass to AgentC_Analyzer

    return ReplyResult(
        message=f"Retrieved {len(results)} materials from Materials Project.",
        target=AgentNameTarget("AgentC_Analyzer"),
        context_variables=context_variables,
    )



# Tool C — final_conclusion

@tool(
    name="final_conclusion_tool",
    description="Produce a structured, multi-dimensional analysis of retrieved materials."
)
def final_conclusion_tool(
    mp_results: Annotated[list, "List of Materials Project documents to analyze."],
    context_variables: ContextVariables,
) -> ReplyResult:

    error_target = AgentNameTarget("AgentC_Analyzer")

    # Validation
    try:
        if not isinstance(mp_results, list):
            raise ValueError("mp_results must be a list.")
        if len(mp_results) == 0:
            raise ValueError("mp_results is empty. No materials to analyze.")
    except ValueError as e:
        return ReplyResult(
            message=f"Error in final_conclusion_tool: {e}",
            target=error_target,
            context_variables=context_variables,
        )

    structured_output = []

    for entry in mp_results:
        material_block = {
            "Material": {
                "ID": entry.get("material_id", None),
                "Formula": entry.get("formula_pretty", None),
                "Bandgap": entry.get("band_gap", None),
                "Density": entry.get("density", None),
                "Volume": entry.get("volume", None),
                "Energy_above_hull": entry.get("e_above_hull", None),
            },
            "Usage": {
                "Applications": "Potential applications based on retrieved properties.",
                "Industry_relevance": "Why this material could matter for industry.",
            },
            "Sustainability": {
                "Abundance": "Placeholder for abundance / criticality analysis.",
                "Environmental_impact": "Placeholder evaluation of toxicity / extraction cost.",
                "Recyclability": "Placeholder for recyclability commentary.",
            },
            "Weaknesses": {
                "Structural_limitations": "Placeholder for mechanical or thermal limitations.",
                "Economic_limitations": "Placeholder for cost or manufacturability issues.",
            }
        }

        structured_output.append(material_block)

    # Save to context variables
    context_variables["final_conclusion"] = structured_output

    return ReplyResult(
        message=structured_output,
        target=AgentNameTarget("Human"),
        context_variables=context_variables,
    )



# Tool D — python_code_block 


project_folder = os.path.abspath("ag2_project")
os.makedirs(project_folder, exist_ok=True)

from autogen.coding.local_commandline_code_executor import LocalCommandLineCodeExecutor
from autogen.coding.base import CodeBlock
from pydantic import BaseModel, Field

class PythonCode(BaseModel):
    code: str = Field(..., description="Full python code executed by the coder agent")


def python_code_block(
    code: str,
    code_file_name: str,
    context_variables: ContextVariables,
):
    """
    The *real* execution engine. Executes Python code, manages context_variables_data.json,
    saves executed code, handles errors, and returns a ReplyResult-compatible object.
    """

    messages = ""
    full_code = '#!/usr/bin/env python3\n' + code
    python_code_model = PythonCode(code=full_code)

   
    # 1. Execute the code
  
    executor = LocalCommandLineCodeExecutor(timeout=60000)
    code_block = CodeBlock(language="python", code=full_code)
    result = executor.execute_code_blocks([code_block])
    exit_code = result.exit_code
    message = result.output

 
    # 2. Load or initialize context file
    
    context_path = os.path.join(project_folder, "context_variables_data.json")

    try:
        with open(context_path, "r") as f:
            context_file_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        context_file_data = {
            "execution_results": {},
            "execution_history": [],
            "execution_notes": []
        }


    # 3. Update context based on exit_code
   
    if exit_code != 0:
        context_file_data["code_error"] = python_code_model.model_dump()
    else:
        if "code" not in context_file_data or not isinstance(context_file_data["code"], list):
            context_file_data["code"] = []
        context_file_data["code"].append(python_code_model.model_dump())
        context_file_data["code_error"] = ""

   
    # 4. Save executed code permanently
    
    code_output_path = os.path.join(project_folder, code_file_name)
    os.makedirs(os.path.dirname(code_output_path), exist_ok=True)

    with open(code_output_path, "w") as f:
        f.write(full_code)

    # Remove temp file created by LocalCommandLineCodeExecutor
    try:
        os.remove(result.code_file)
    except Exception:
        pass

    
    # 5. Save context json
   
    with open(context_path, "w") as f:
        json.dump(context_file_data, f, indent=2)

   
    # 6. Build the return message
  
    if exit_code == 0:
        messages += f"Code executed successfully.\nOutput:\n{message}\n\n"
        messages += "Proceed with the next python_code_block call if needed."
    else:
        messages += f"Code execution failed.\nError:\n{message}\n\n"
        messages += "Correct the code and try again."

   
    return ReplyResult(
        message=messages,
        target=AgentNameTarget("AgentD_Coder"),
        context_variables=context_variables,
    )



# Tool wrapper for AG2


@tool(
    name="python_code_block_tool",
    description="Execute Python code and save results into the project folder."
)
def python_code_block_tool(
    code: Annotated[str, "A single block of Python code to execute."],
    file_name: Annotated[str, "Name of the code file to store the executed script, e.g., 'script.py'"],
    context_variables: ContextVariables,
) -> ReplyResult:

    error_target = AgentNameTarget("AgentD_Coder")

    # Validation
    try:
        if not isinstance(code, str):
            raise ValueError("code must be a string.")
        if not isinstance(file_name, str):
            raise ValueError("file_name must be a string.")
        if not file_name.strip():
            raise ValueError("file_name cannot be empty.")
    except ValueError as e:
        return ReplyResult(
            message=f"Error in python_code_block_tool: {e}",
            target=error_target,
            context_variables=context_variables,
        )

    # Delegate to real executor
    return python_code_block(
        code=code,
        code_file_name=file_name,
        context_variables=context_variables,
    )

# Tool E — save_code_to_file

@tool(
    name="save_code_to_file_tool",
    description="Save a Python code string to a file inside the project folder."
)
def save_code_to_file_tool(
    code: Annotated[str, "Python code string to save to a file."],
    file_name: Annotated[str, "Name of the file to create inside the project folder."],
    context_variables: ContextVariables,
) -> ReplyResult:

    error_target = AgentNameTarget("AgentD_Coder")

    # Validation
    try:
        if not isinstance(code, str):
            raise ValueError("The 'code' parameter must be a string.")
        if not isinstance(file_name, str):
            raise ValueError("The 'file_name' parameter must be a string.")
        if len(file_name.strip()) == 0:
            raise ValueError("file_name cannot be empty.")
    except ValueError as e:
        return ReplyResult(
            message=f"Error in save_code_to_file_tool: {e}",
            target=error_target,
            context_variables=context_variables,
        )

    # Compute the path
    file_path = os.path.join(project_folder, file_name)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Write the file
    with open(file_path, "w") as f:
        f.write(code)

    message = f"Code saved to: {file_path}"

    return ReplyResult(
        message=message,
        target=error_target,
        context_variables=context_variables,
    )

    # Compute the path
    file_path = os.path.join(project_folder, file_name)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Write the file
    with open(file_path, "w") as f:
        f.write(code)


# 5. Manual tests for Materials Project tool


In [26]:
print("Running manual tests for download_materials_structures_properties_from_mp...\n")

from autogen.agentchat.group import ContextVariables

# Test 1 — Simple valid query
try:
    search_criteria_1 = {
        "band_gap": (3, 5),
        "energy_above_hull": (0, 0.1),
    }
    fields_1 = ["material_id", "formula_pretty", "band_gap", "energy_above_hull"]
    sample_number_1 = 5

    ctx_1 = ContextVariables(data={})
    result_1 = download_materials_structures_properties_from_mp(
        search_criteria=search_criteria_1,
        fields=fields_1,
        sample_number=sample_number_1,
        context_variables=ctx_1,
    )
    print("Test 1 passed:", result_1.message)
except Exception as e:
    print("Test 1 failed with unexpected exception:", e)


# Test 2 — Invalid search_criteria content 
try:
    # Empty dict: correct type, pero tu tool lo rechaza por estar vacío
    search_criteria_2 = {}
    fields_2 = ["material_id"]
    sample_number_2 = 3

    ctx_2 = ContextVariables(data={})
    result_2 = download_materials_structures_properties_from_mp(
        search_criteria=search_criteria_2,
        fields=fields_2,
        sample_number=sample_number_2,
        context_variables=ctx_2,
    )
    if "Error in download_materials_structures_properties_from_mp" in result_2.message:
        print("Test 2 passed (expected error):", result_2.message)
    else:
        print("Test 2 FAILED (no validation error):", result_2.message)
except Exception as e:
    print("Test 2 failed with unexpected exception:", e)


# Test 3 — Empty fields list
try:
    search_criteria_3 = {
        "band_gap": (1, 3),
    }
    fields_3 = []
    sample_number_3 = 3

    ctx_3 = ContextVariables(data={})
    result_3 = download_materials_structures_properties_from_mp(
        search_criteria=search_criteria_3,
        fields=fields_3,
        sample_number=sample_number_3,
        context_variables=ctx_3,
    )
    if "Error in download_materials_structures_properties_from_mp" in result_3.message:
        print("Test 3 passed (expected error):", result_3.message)
    else:
        print("Test 3 FAILED (no validation error):", result_3.message)
except Exception as e:
    print("Test 3 failed with unexpected exception:", e)


# Test 4 — Invalid sample_number (<= 0)
try:
    search_criteria_4 = {
        "band_gap": (0, 10),
    }
    fields_4 = ["material_id"]
    sample_number_4 = 0

    ctx_4 = ContextVariables(data={})
    result_4 = download_materials_structures_properties_from_mp(
        search_criteria=search_criteria_4,
        fields=fields_4,
        sample_number=sample_number_4,
        context_variables=ctx_4,
    )
    if "Error in download_materials_structures_properties_from_mp" in result_4.message:
        print("Test 4 passed (expected error):", result_4.message)
    else:
        print("Test 4 FAILED (no validation error):", result_4.message)
except Exception as e:
    print("Test 4 failed with unexpected exception:", e)


# Test 5 — Invalid filter key
try:
    search_criteria_5 = {
        "invalid_property": (0, 1),
    }
    fields_5 = ["material_id"]
    sample_number_5 = 3

    ctx_5 = ContextVariables(data={})
    result_5 = download_materials_structures_properties_from_mp(
        search_criteria=search_criteria_5,
        fields=fields_5,
        sample_number=sample_number_5,
        context_variables=ctx_5,
    )
    if "Error in download_materials_structures_properties_from_mp" in result_5.message:
        print("Test 5 passed (expected error):", result_5.message)
    else:
        print("Test 5 FAILED (no validation error):", result_5.message)
except Exception as e:
    print("Test 5 failed with unexpected exception:", e)


print("\nManual tool tests completed.")



Running manual tests for download_materials_structures_properties_from_mp...



Retrieving SummaryDoc documents:   0%|          | 0/14851 [00:00<?, ?it/s]

Test 1 passed: Retrieved 5 materials from Materials Project.
Test 2 passed (expected error): Error in download_materials_structures_properties_from_mp: search_criteria cannot be empty.
Test 3 passed (expected error): Error in download_materials_structures_properties_from_mp: fields list cannot be empty.
Test 4 passed (expected error): Error in download_materials_structures_properties_from_mp: sample_number must be a positive integer.
Test 5 passed (expected error): Error in download_materials_structures_properties_from_mp: invalid filter 'invalid_property'. Allowed filters: ['band_gap', 'chemsys', 'elements', 'energy_above_hull', 'excluded_elements', 'g_voigt', 'k_voigt', 'num_sites']

Manual tool tests completed.


# 6. AGENTS (A → B → C → D + HUMAN)

In [27]:
# --- Agent A: Explainer ---

explainer_message = """
You are AgentA_Explainer.

### ROLE
You interpret the user's materials-science question and produce a structured scientific explanation.

### YOUR TASKS
- Parse the user's query from context_variables["user_query"].
- Identify relevant scientific terminology (e.g., bandgap, symmetry, density, conductivity).
- Generate a clear explanation of what the user is asking.
- Clarify key terms in simple but technically correct language.

### TOOL YOU MUST USE
- You MUST call the tool: explain_query_tool
- Never produce free text.
- The explanation MUST be generated ONLY through explain_query_tool.

### HOW TO CALL THE TOOL (IMPORTANT)
- You must call explain_query_tool with EXACTLY TWO arguments:
  - "query": the original user query string taken from context_variables["user_query"].
  - "query_explanation": your scientific explanation as a string.
- Do NOT include "context_variables" in the arguments. It is automatically provided by the system.

The JSON arguments MUST look like this:
{
  "query": "<original user query>",
  "query_explanation": "<your detailed scientific explanation>"
}

### FORBIDDEN ACTIONS
- Do NOT solve the materials-science query.
- Do NOT retrieve materials.
- Do NOT analyze data.
- Do NOT talk without using the tool.
"""

AgentA_Explainer = ConversableAgent(
    name="AgentA_Explainer",
    llm_config=llm_config,
    system_message=explainer_message,
    human_input_mode="NEVER",
    functions=[explain_query_tool],
)



# --- Agent B — Materials Retriever ---

retriever_message = """
You are AgentB_MaterialsRetriever.

### ROLE
You transform the explained query into a valid Materials Project query.

### YOUR TASKS
- Read the explanation from context_variables["explained_terms"].
- Construct a valid Materials Project API search request using search_criteria + fields + sample_number.
- Validate the query structure before calling the tool.
- Use the tool to fetch materials.

### TOOL YOU MUST USE
- download_materials_structures_properties_from_mp
- You cannot talk without calling the tool.

### MUST VALIDATE BEFORE TOOL CALL
- Ensure search_criteria is a dict.
- Ensure fields is a non-empty list.
- Ensure sample_number is a positive integer.
- Raise errors for invalid keys, e.g.:
  - Allowed numeric filters: band_gap, energy_above_hull, k_voigt, g_voigt, num_sites
  - Allowed composition filters: elements, chemsys, excluded_elements
- Raise errors for missing or empty fields.

### OUTPUT BEHAVIOR
- You MUST call the tool: download_materials_structures_properties_from_mp
- The tool output will automatically update context_variables (e.g. mp_results)
  and route to AgentC_Analyzer.
- Do NOT produce any JSON or free text directly.

### FORBIDDEN ACTIONS
- No free text.
- No analysis (belongs to Agent C).
- No coding (belongs to Agent D).
"""

AgentB_MaterialsRetriever = ConversableAgent(
    name="AgentB_MaterialsRetriever",
    llm_config=llm_config,
    system_message=retriever_message,
    human_input_mode="NEVER",
    functions=[download_materials_structures_properties_from_mp],
)



# --- Agent C — Analyzer ---

analyzer_message = """
You are AgentC_Analyzer.

### ROLE
You analyze retrieved Materials Project results stored in context_variables["mp_results"].

### YOUR TASKS
- Read MP results and extract chemical and physical information.
- Organize material data into a highly structured scientific summary.
- Produce a final human-readable conclusion via the tool final_conclusion_tool.

### REQUIRED OUTPUT STRUCTURE
For each material produce:

Material:
- ID:
- Formula:
- Bandgap:
- Density:
- Volume:
- Energy_above_hull:
Usage:
Sustainability:
Weaknesses:

### TOOL YOU MUST USE
- final_conclusion_tool
- You MUST produce all output using the tool.
- No direct text output.

### FORBIDDEN ACTIONS
- Do NOT retrieve materials (belongs to Agent B)
- Do NOT explain user queries (belongs to Agent A)
- Do NOT execute Python (belongs to Agent D)
"""

AgentC_Analyzer = ConversableAgent(
    name="AgentC_Analyzer",
    llm_config=llm_config,
    system_message=analyzer_message,
    human_input_mode="NEVER",
    functions=[final_conclusion_tool],
)



# --- Agent D — Coding Agent ---

coder_message = """
You are AgentD_Coder.

### ROLE
Generate and execute Python code when requested by the other agents.

### YOUR TASKS
- Write Python functions, calculations, or plots.
- Execute Python code strictly through the python_code_block_tool.
- Save executed code into ag2_project/<file_name>.
- Return only clean, concise execution results (via the tool output).

### TOOL YOU MUST USE
- python_code_block_tool
- You MUST always call python_code_block_tool to run any code.
- You must NEVER return raw Python code in the chat.

### FORBIDDEN ACTIONS
- Never produce plain Python code in the chat.
- Never answer scientific questions.
- Never retrieve materials.
"""

AgentD_Coder = ConversableAgent(
    name="AgentD_Coder",
    llm_config=llm_config,
    system_message=coder_message,
    human_input_mode="NEVER",
    functions=[python_code_block_tool],
)



# --- Human Agent ---

Human = ConversableAgent(
    name="Human",
    human_input_mode="ALWAYS",
)


# 7. Pattern definition 


In [24]:
# --- Pattern: next speaker logic ---

def select_next_speaker(last_speaker, groupchat):
    # First turn → Human speaks
    if last_speaker is None:
        return groupchat.agent_lookup["Human"]

    # Human → AgentA_Explainer
    if last_speaker.name == "Human":
        return groupchat.agent_lookup["AgentA_Explainer"]

    # AgentA_Explainer → AgentB_MaterialsRetriever
    if last_speaker.name == "AgentA_Explainer":
        return groupchat.agent_lookup["AgentB_MaterialsRetriever"]

    # AgentB_MaterialsRetriever → AgentC_Analyzer
    if last_speaker.name == "AgentB_MaterialsRetriever":
        return groupchat.agent_lookup["AgentC_Analyzer"]

    # AgentC_Analyzer → Human
    # (Si AgentC necesita código, lo pedirá explícitamente vía tool a AgentD_Coder;
    # este patrón solo define el flujo "normal".)
    if last_speaker.name == "AgentC_Analyzer":
        return groupchat.agent_lookup["Human"]

    # AgentD_Coder → Human
    if last_speaker.name == "AgentD_Coder":
        return groupchat.agent_lookup["Human"]

    # Fallback
    return groupchat.agent_lookup["Human"]


# --- GroupChat setup ---

group_chat = GroupChat(
    agents=[
        Human,
        AgentA_Explainer,
        AgentB_MaterialsRetriever,
        AgentC_Analyzer,
        AgentD_Coder,
    ],
    messages=[],
    max_round=20,
    speaker_selection_method=select_next_speaker,
)

# Required lookup table for AG2
group_chat.agent_lookup = {agent.name: agent for agent in group_chat.agents}

# Attach context variables
group_chat.context_variables = context_variables

# Manager
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

print("Pattern loaded.")



Pattern loaded.


# 8. Group Chat

In [22]:
print("\n=== WildfiresAI | Multi-Agent Materials System ===\n")

Human.initiate_chat(
    manager,
    message="Please enter your materials-science question.",
    context_variables=context_variables,
)





=== WildfiresAI | Multi-Agent Materials System ===

Human (to chat_manager):

Please enter your materials-science question.

--------------------------------------------------------------------------------

Next speaker: AgentA_Explainer

AgentA_Explainer (to chat_manager):

***** Suggested tool call (call_ZESfpp750wBUNfyVgz9A0yEL): explain_query_tool *****
Arguments: 
{"query":"What is the effect of temperature on the bandgap of semiconductors?","query_explanation":"The user is asking about how temperature variations influence the bandgap of semiconductor materials. The bandgap is the energy difference between the top of the valence band and the bottom of the conduction band in a semiconductor. It is crucial for determining a material's electrical conductivity. Generally, as temperature increases, the bandgap tends to decrease due to increased atomic vibrations affecting electron energy levels. This understanding is key in applications like electronics and photovoltaics, where temper

Replying as Human. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  material for fire in forest


Human (to chat_manager):

material for fire in forest

--------------------------------------------------------------------------------

Next speaker: AgentA_Explainer

AgentA_Explainer (to chat_manager):

***** Suggested tool call (call_0nP8OGW2gu04MDbxqMhLgCps): explain_query_tool *****
Arguments: 
{"query":"material for fire in forest","query_explanation":"The user is likely inquiring about the materials that influence or are affected by fires in forest environments. This could involve aspects such as:\n\n- **Flammability of Forest Materials**: Understanding which materials (like different types of wood or vegetation) easily catch fire or sustain it. This relates to the study of biomass and organic materials.\n\n- **Fire-resistant Materials**: Materials used to prevent or control the spread of fire, including natural barriers or man-made interventions like fire retardants.\n\n- **Impact of Fire on Soil and Vegetation**: Considering changes in chemical composition and physical proper

Replying as Human. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (7be4e650-c182-4705-bf61-1573b5cb6843): User requested to end the conversation

>>>>>>>> TERMINATING RUN (bb1a84ee-04f7-4dd0-82c7-d7c2c2d01243): No reply generated


ChatResult(chat_id=56016441849502169857820810710390987230, chat_history=[{'content': 'Please enter your materials-science question.', 'role': 'assistant', 'name': 'Human'}, {'content': 'None', 'name': 'AgentA_Explainer', 'tool_calls': [{'id': 'call_ZESfpp750wBUNfyVgz9A0yEL', 'function': {'arguments': '{"query":"What is the effect of temperature on the bandgap of semiconductors?","query_explanation":"The user is asking about how temperature variations influence the bandgap of semiconductor materials. The bandgap is the energy difference between the top of the valence band and the bottom of the conduction band in a semiconductor. It is crucial for determining a material\'s electrical conductivity. Generally, as temperature increases, the bandgap tends to decrease due to increased atomic vibrations affecting electron energy levels. This understanding is key in applications like electronics and photovoltaics, where temperature effects can influence semiconductor performance."}', 'name': 'e